In [1]:
import pandas as pd
import numpy as np
import re
import openpyxl
import easygui
import os

In [2]:
#仓库销售编码字典
fname0 =  r"F:\a00nutstore\008\zw08\用友报价\仓库销售编码字典.xlsx"
df0 = pd.read_excel(fname0)
dic_ck_xs = dict(zip(df0['仓库货号'],df0['销售货号']))
dic_xs_ck = dict(zip(df0['销售货号'],df0['仓库货号']))


In [3]:
#产成品仓库库存
def chuliChangku(fname,sheet_name):
    df = pd.read_excel(fname,sheet_name = sheet_name)
    #删除第一列的空行
    df = df[~df['类别'].isna()]
    df = df[~df['类别'].str.contains('小计|累计|合   计')]
    df = df[~(abs(df['本日数量'] - 0) < 0.00001)]
    return df
fname = r"F:\a00nutstore\008\zw08\用友报价\2024.7.16电商仓库数据 - 副本.xls"
sheet_name  =  0
df = chuliChangku(fname,sheet_name)
df
df.to_excel('11.xlsx',index = False)




In [4]:
def addXiaoshouhuohao(df,dic_ck_xs):
    df_ck = df.assign(销售货号 = df['货号'].map(dic_ck_xs))
    gp = df_ck.groupby('货号')
    df_ck = pd.DataFrame(gp.sum())
    df_ck = df_ck[['销售货号','本日数量']]
    df_ck = df_ck.reset_index()
    return df_ck
df_ck = addXiaoshouhuohao(df,dic_ck_xs)
df_ck = df_ck[['本日数量','销售货号','货号']]
df_ck = df_ck.rename(columns = {'货号':'仓库货号'})
df_ck.to_excel('仓库库存-电商.xlsx',index = False)  #打开文件后，要确保销售货号不为空

df_ck


,本日数量,销售货号,仓库货号
0,1,1101,1101
1,1,1103,1103
2,22,1121,1121
3,1,1125,1125
4,310,2001,2001
...,...,...,...
146,20,RFDS-1660N,RFDS-1660N
147,-10,RFDS-3260N,RFDS-3260N
148,-10,RFJC-3260N,RFJC-3260N
149,640,RFSX-3260N,RFSX-3260N


In [73]:
#销售编码
def xiaoshou(fname_xs):
    df_xs = pd.read_excel(fname_xs)
    df_xs = df_xs[~df_xs['类别'].isna()]
    return df_xs

        
fname_xs = r"F:\a00nutstore\008\zw08\用友报价\销售仓库价格库存1.xlsx"
xs_lst = [
 '类别',
 '货号',
 '品名',
 '销售部存货大类编码',
 '含量',
 '汉办',
 '汉口北电商',
 '外地',
'仓库货号'
]
df_xs = pd.read_excel(fname_xs)
df_xs = df_xs[xs_lst ]
# df_xs = df_xs.assign(仓库货号 = df_xs['货号'].map(dic_ck_xs))


In [ ]:
#新增仓库货号
lst = set(df_ck['仓库货号'])-set(df_xs['货号'])
lst


In [ ]:
df_ck_diff = pd.DataFrame({'仓库货号':list(lst)})
df_ck_diff

In [74]:
#销售货号唯一化处理
def weiyihuXiaoshouhuohao(df_xs,df_ck,dic_xs_ck):
    merge = pd.merge(df_xs,df_ck,left_on = '货号',right_on = '销售货号',how = 'right')
    merge['本日数量本'] = merge['本日数量'] 
    merge['本日数量件'] = merge['本日数量'] /merge['含量']
    del merge['本日数量']
    # del merge['销售货号']
    gp = merge.groupby('销售货号')
    df0 = pd.DataFrame(gp)
    dic_agg = {'含量':'mean','汉办':'mean','汉口北电商':'mean','外地':'mean','本日数量本':'sum'}

    right = df0.agg(dic_agg)
    right = right.reset_index()
    left = df_xs[[
 '类别',
 '货号',
 '品名',
 '销售部存货大类编码'
]]
    mg  = pd.merge(left,right,left_on = '货号',right_on = '销售货号',how = 'right')
    mg['本日数量件'] = mg['本日数量本']/mg['含量']
    mg = mg.assign(仓库货号 = mg['货号'].map(dic_xs_ck))
    return mg

result = weiyihuXiaoshouhuohao(df_xs,df_ck,dic_xs_ck)
result 
    
        

KeyError: "Column(s) ['含量', '外地', '本日数量本', '汉办', '汉口北电商'] do not exist"

In [47]:
result.to_excel('销售仓库价格库存0716.xlsx',index = False)

C:\Users\redda\AppData\Local\Temp\ipykernel_13684\4220249614.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  result.to_excel('销售仓库价格库存0716.xlsx',index = False)


In [ ]:
df_ck0 = chuliChangku(fname,sheet_name = 'AAA')
df_ck0

In [ ]:
df_ck0 = df_ck0[['本日数量','销售货号','货号']]
df_ck0 = df_ck0.rename(columns = {'货号':'仓库货号'})
df_ck0

In [ ]:
df_ck0

In [ ]:
dic_ck_xs

In [ ]:
df_ck0 = addXiaoshouhuohao(df_ck0,dic_ck_xs)
df_ck0

In [ ]:
def cangkuDiff(df0,df1):
    df0 = df0.rename(columns = {'货号':'仓库货号'})
    df1 = df1.rename(columns = {'货号':'仓库货号'})
    df0 = df0[['货号','本日数量']]
    df1 = df1[['货号','本日数量']]
    merge = pd.merge(df0,df1,left_on= '仓库货号',right_on= '仓库货号',how = 'outer')
    return merge
diff = cangkuDiff(df_ck0,df_ck)
diff
    
    

In [ ]:
df_ck0

In [ ]:
df_ck

In [ ]:
df_ck0 = df_ck0.rename(columns = {'货号':'仓库货号'})
df_ck0

In [ ]:
df_ck = df_ck.rename(columns = {'货号':'仓库货号'})
df_ck

In [ ]:
merge = pd.merge(df_ck0,df_ck,left_on= '仓库货号',right_on= '仓库货号',how = 'outer')
merge

In [ ]:
merge['本日数量_y'] = merge['本日数量_y'].fillna(0)
merge['本日数量_x'] = merge['本日数量_x'].fillna(0)
merge

In [ ]:
merge['diff'] = merge['本日数量_y']- merge['本日数量_x']
merge

In [ ]:
merge.to_excel('9999.xlsx',index = False)

In [ ]:
merge['销售货号_x'] =  merge['仓库货号'].map(dic_ck_xs)
merge

In [ ]:
del merge['销售货号_y']
merge = merge.rename({'销售货号_x':'销售货号'})
merge.to_excel('9999.xlsx',index = False)


In [ ]:
df_ck_diff.to_excel('110.xlsx',index = False)

In [ ]:
df.to_excel('11.xlsx',index = False)

In [ ]:
df_ck

In [ ]:
merge = pd.merge(df_xs,df_ck,left_on = '货号',right_on = '仓库货号',how = 'right')
merge['本日数量本'] = merge['本日数量'] 
merge['本日数量件'] = merge['本日数量'] /merge['含量']
del merge['本日数量']
del merge['销售货号']
gp = merge.groupby('仓库货号')
dic_agg = {'含量':'mean','汉办':'mean','汉口北电商':'mean','外地':'mean','本日数量本':'sum'}
right = gp.agg(dic_agg)
right = right.reset_index()
left = df_xs[[
'类别',
'货号',
'品名',
'销售部存货大类编码'
]]
mg  = pd.merge(left,right,left_on = '货号',right_on = '销售货号',how = 'right')
# mg['本日数量件'] = mg['本日数量本']/mg['含量']
# mg = mg.assign(仓库货号 = mg['货号'].map(dic_xs_ck))
# return mg

# result = weiyihuXiaoshouhuohao(df_xs,df_ck,dic_xs_ck)
# result 
merge



In [26]:
merge = pd.merge(df_xs,df_ck,left_on = '货号',right_on = '销售货号',how = 'right')
merge['本日数量本'] = merge['本日数量'] 
merge['本日数量件'] = merge['本日数量'] /merge['含量']
del merge['本日数量']
# del merge['销售货号']
gp = merge.groupby('销售货号')
dic_agg = {'含量':'mean','汉办':'mean','汉口北电商':'mean','外地':'mean','本日数量本':'sum'}
right = gp.agg(dic_agg)
right = right.reset_index()
right

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
right.to_excel('333.xlsx',index = False)

In [ ]:
left = df_xs[[
'类别',
'货号',
'品名',
'销售部存货大类编码'
]]
mg  = pd.merge(left,right,left_on = '货号',right_on = '销售货号',how = 'right')
mg.to_excel('444.xlsx',index= False)

In [32]:
merge = pd.merge(df_xs,df_ck,left_on = '货号',right_on = '销售货号',how = 'right')
merge['本日数量本'] = merge['本日数量'] 
merge['本日数量件'] = merge['本日数量'] /merge['含量']
merge

,类别,货号,品名,销售部存货大类编码,含量,汉办,汉口北电商,外地,本日数量,销售货号,仓库货号,本日数量本,本日数量件
0,普通账芯,1101,16K总分类账,01010201,60.0,5.52,5.5200,5.5200,2220,1101,1101,2220,37.0
1,普通账芯,1103,16K银行存款日记账,01010201,60.0,5.52,5.5200,5.5200,900,1103,1103,900,15.0
2,普通账芯,1105,16K现金日记账,01010201,60.0,5.52,5.5200,5.5200,900,1105,1105,900,15.0
3,普通账芯,1106,16K三栏式明细分类账,01010201,60.0,5.52,5.5200,5.5200,7080,1106,1106,7080,118.0
4,普通账芯,1107,16K固定资产明细分类账,01010201,60.0,5.52,5.5200,5.5200,1140,1107,1107,1140,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1582,外贸订制品,瑞典A460g线条,瑞典A460g线条,010214,40.0,0,0.0000,0.0000,2240,瑞典A460g线条,瑞典A460g线条,2240,56.0
1583,外贸订制品,瑞典A470g线条,瑞典A470g线条,010214,40.0,0,0.0000,0.0000,40,瑞典A470g线条,瑞典A470g线条,40,1.0
1584,外贸订制品,美国A6铁订本*,美国A6铁订本(枫叶)*,010214,36.0,0,0.0000,0.0000,252,美国A6铁订本*,美国A6铁订本(枫叶)*,252,7.0
1585,外贸订制品,美国A6铁订本(枫叶)*,美国A6铁订本(枫叶)*,010214,36.0,0,0.0000,0.0000,324,美国A6铁订本(枫叶)*,美国A6铁订本*,324,9.0


In [33]:
# del merge['本日数量']
# del merge['销售货号']
gp = merge.groupby('销售货号')
gp
# dic_agg = {'含量':'mean','汉办':'mean','汉口北电商':'mean','外地':'mean','本日数量本':'sum'}
# right = gp.agg(dic_agg)
# right = right.reset_index()
# left = df_xs[[
# '类别',
# '货号',
# '品名',
# '销售部存货大类编码'
# ]]
# mg  = pd.merge(left,right,left_on = '货号',right_on = '销售货号',how = 'right')
# mg['本日数量件'] = mg['本日数量本']/mg['含量']
# mg = mg.assign(仓库货号 = mg['货号'].map(dic_xs_ck))

In [35]:
dic_agg = {'含量':'mean','汉办':'mean','汉口北电商':'mean','外地':'mean','本日数量本':'sum'}
right = gp.agg(dic_agg)
right = right.reset_index()
# left = df_xs[[
# '类别',
# '货号',
# '品名',
# '销售部存货大类编码'
# ]]
# mg  = pd.merge(left,right,left_on = '货号',right_on = '销售货号',how = 'right')
# mg['本日数量件'] = mg['本日数量本']/mg['含量']
# mg = mg.assign(仓库货号 = mg['货号'].map(dic_xs_ck))

TypeError: agg function failed [how->mean,dtype->object]

In [37]:
df_xs.to_excel('0001.xlsx',index= False)

C:\Users\redda\AppData\Local\Temp\ipykernel_13684\3438724983.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_xs.to_excel('0001.xlsx',index= False)


In [39]:
df_xs.to_excel('0002.xlsx',index = False)

C:\Users\redda\AppData\Local\Temp\ipykernel_13684\1232360964.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_xs.to_excel('0002.xlsx',index = False)


In [48]:
df01 = pd.read_clipboard()
df01

,销售货号
0,5514
1,7005
2,A1-054197
3,A4单线信纸
4,CQL-1660-牛卡
...,...
63,XJ401S
64,XQ106
65,YTY1660
66,YTY1680


In [53]:
df_xs

,类别,货号,品名,销售部存货大类编码,含量,汉办,汉口北电商,外地
0,多联单据,5007,48K75P送货单(三联、多栏),01010303,360.0,1.48,1.4800,1.4800
1,多联单据,5008,48K75P缴款单(三联、多栏),01010303,360.0,1.48,1.4800,1.4800
2,多联单据,5009,48K75P收据(三联、单栏),01010303,360.0,1.48,1.4800,1.4800
3,多联单据,5014,40K75P菜单(三联、竖式多栏),01010303,360.0,1.48,1.4800,1.4800
4,多联单据,5017,32K75P送货单(三联、竖式多栏),01010303,240.0,1.93,1.9300,1.9300
...,...,...,...,...,...,...,...,...
1534,外贸订制品,瑞典A460g方格,瑞典A460g方格,010214,40.0,0,0.0000,0.0000
1535,外贸订制品,瑞典A460g线条,瑞典A460g线条,010214,40.0,0,0.0000,0.0000
1536,外贸订制品,瑞典A470g线条,瑞典A470g线条,010214,40.0,0,0.0000,0.0000
1537,装订配件,商品卡,48K商品卡,010106,5400.0,0.11,0.1100,0.1100


In [54]:
df02 = pd.merge(df01,df_xs,how='left',left_on = '销售货号',right_on = '货号')
df02

,销售货号,类别,货号,品名,销售部存货大类编码,含量,汉办,汉口北电商,外地
0,5514,多联单据,5514,32K75P入库单(三联、多栏),01010303,240.0,1.93,1.93,1.93
1,7005,报表,7005,16K工资表（横式）,010105,220.0,1.37,1.37,1.37
2,A1-054197,Y出口订制产品,A1-054197,A1-054197,010214,NaN,NaN,NaN,NaN
3,A4单线信纸,订制产品,A4单线信纸,A4单线信纸,010211,300.0,0.93,0.93,0.93
4,CQL-1660-牛卡,订制产品,CQL-1660-牛卡,16K60型牛卡软抄,010211,200.0,1.22,1.22,1.22
...,...,...,...,...,...,...,...,...,...
63,XJ401S,新疆版小学生本,XJ401S,16K精品牛卡英语缝线本,01020809,280.0,0.86,0.86,0.86
64,XQ106,天津南开西青区小字本,XQ106,抄书本,01020818,550.0,0.3,0.30,0.30
65,YTY1660,订制产品,YTY1660,16K60型牛皮英语本,010211,200.0,1.13,1.13,1.13
66,YTY1680,订制产品,YTY1680,16K80型牛皮英语本,010211,140.0,1.49,1.49,1.49


In [55]:
df02.to_excel('0003.xlsx',index = False)

C:\Users\redda\AppData\Local\Temp\ipykernel_13684\224802611.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df02.to_excel('0003.xlsx',index = False)


In [66]:
merge = pd.merge(df_xs,df_ck,left_on = '货号',right_on = '销售货号',how = 'right')
merge['本日数量本'] = merge['本日数量'] 
merge['本日数量件'] = merge['本日数量'] /merge['含量']
del merge['本日数量']
# del merge['销售货号']
gp = merge.groupby('销售货号')
gp
# df0 = pd.DataFrame(gp)


In [67]:
right = gp.agg(dic_agg)
right

TypeError: agg function failed [how->mean,dtype->object]

In [68]:
merge.to_excel('0004.xlsx',index = False)

C:\Users\redda\AppData\Local\Temp\ipykernel_13684\4016668095.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  merge.to_excel('0004.xlsx',index = False)


In [75]:
df_xs

,类别,货号,品名,销售部存货大类编码,含量,汉办,汉口北电商,外地,仓库货号
0,多联单据,5007,48K75P送货单(三联、多栏),01010303,360,1.48,1.48,1.48,5007A
1,多联单据,5008,48K75P缴款单(三联、多栏),01010303,360,1.48,1.48,1.48,5008A
2,多联单据,5009,48K75P收据(三联、单栏),01010303,360,1.48,1.48,1.48,5009A
3,多联单据,5014,40K75P菜单(三联、竖式多栏),01010303,360,1.48,1.48,1.48,5014A
4,多联单据,5017,32K75P送货单(三联、竖式多栏),01010303,240,1.93,1.93,1.93,5017A
...,...,...,...,...,...,...,...,...,...
1575,新疆版小学生本,XJ401S,XJ401S,16K精品牛卡英语缝线本,1020809,280.00,0.86,0.86,XJ401S
1576,天津南开西青区小字本,XQ106,XQ106,抄书本,1020818,550.00,0.30,0.30,XQ106
1577,订制产品,YTY1660,YTY1660,16K60型牛皮英语本,10211,200.00,1.13,1.13,YTY1660
1578,订制产品,YTY1680,YTY1680,16K80型牛皮英语本,10211,140.00,1.49,1.49,YTY1680


In [76]:
df_ck

,本日数量,销售货号,仓库货号
0,2220,1101,1101
1,900,1103,1103
2,900,1105,1105
3,7080,1106,1106
4,1140,1107,1107
...,...,...,...
1582,2240,瑞典A460g线条,瑞典A460g线条
1583,40,瑞典A470g线条,瑞典A470g线条
1584,252,美国A6铁订本*,美国A6铁订本(枫叶)*
1585,324,美国A6铁订本(枫叶)*,美国A6铁订本*


In [5]:
df0 = pd.read_clipboard()
df0

,本日数量,销售货号,仓库货号
0,86040,5007,5007A
1,9000,5008,5008A
2,129600,5009,5009A
3,21960,5014,5014A
4,29520,5017,5017A
...,...,...,...
1583,720,瑞典A460g方格,瑞典A460g方格
1584,2240,瑞典A460g线条,瑞典A460g线条
1585,40,瑞典A470g线条,瑞典A470g线条
1586,113400,商品卡,商品卡


In [8]:
a= df0.groupby('销售货号')['本日数量'].sum()

In [9]:
df1 = pd.DataFrame(a)
df1

,本日数量
销售货号,
1101,2220
1103,900
1105,900
1106,7080
1107,1140
...,...
瑞典A460g线条,2240
瑞典A470g线条,40
美国A6铁订本(枫叶)*,324


In [10]:
df1 = df1.reset_index()

In [11]:
df1

,销售货号,本日数量
0,1101,2220
1,1103,900
2,1105,900
3,1106,7080
4,1107,1140
...,...,...
1529,瑞典A460g线条,2240
1530,瑞典A470g线条,40
1531,美国A6铁订本(枫叶)*,324
1532,美国A6铁订本*,252


In [12]:
df1.to_excel('0001.xlsx',index= False)

In [13]:
df2 = pd.read_clipboard()
df2

,货号
0,1101
1,1103
2,1105
3,1106
4,1107
...,...
1527,瑞典A460g方格
1528,瑞典A460g线条
1529,瑞典A470g线条
1530,商品卡


In [15]:
lst = list(set(df2['货号'])-set(df1['销售货号']))
lst

[]